<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold">
<br>NEXT TONNE - BACKGROUND INDEX</p><br>

In [ ]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# General importings
import math
import numpy             as np
import tables            as tb
import pandas            as pd
import matplotlib.pyplot as plt

# IC stuff
import invisible_cities.core.system_of_units as units

# TONNE stuff
from detector_dimensions  import get_dimensions
from detector_dimensions  import print_dimensions

from material_activities  import get_activities
from material_activities  import print_activities

from detector_backgrounds import get_background_level
from rejection_factors    import get_rejection_factors
from roi_settings         import get_roi_settings


# DETAILS OF ANALYSIS

In [ ]:
# VERBOSITY
VERBOSE = True

# Available detectors: "next_2x2", "next_3x3", "next_hd"
DETECTOR = 'next_hd'

# Available background levels: "reference", "probable", "optimistic"
BKGND_LEVEL = 'reference'

# LOADING DETECTOR DATA

In [ ]:
det_dim = get_dimensions(DETECTOR)

In [ ]:
if VERBOSE:
    print_dimensions(DETECTOR)

In [ ]:
mat_act = get_activities(BKGND_LEVEL)

In [ ]:
if VERBOSE:
    print_activities(BKGND_LEVEL)

# COMPUTING THE DETECTOR BACKGROUND LEVEL

#### Radiogenic contributions

In principle. only the contributions from the 'READOUT_PLANES', 'FIELD_CAGE' and 'INNER_SHIELDING' are considered.

Backgrounds expressed in Becquerel

In [ ]:
radiogenic_bkgnd = get_background_level(DETECTOR, BKGND_LEVEL)
radiogenic_bkgnd['Total'] = radiogenic_bkgnd['Bi214'] + radiogenic_bkgnd['Tl208']
radiogenic_bkgnd

In [ ]:
if VERBOSE:
    tot_bkgnd = radiogenic_bkgnd['Total'].sum()
    print(f"Radiogenic '{DETECTOR}' background: {tot_bkgnd:.4} Bq.")
    
    tot_Bi = radiogenic_bkgnd['Bi214'].sum()
    tot_Tl = radiogenic_bkgnd['Tl208'].sum()
    print(f"Total Bi214 background: {tot_Bi:8.4} Bq.  ->  {tot_Bi/tot_bkgnd*100:.4} %")
    print(f"\nTotal Tl208 background: {tot_Tl:8.4} Bq.  ->  {tot_Tl/tot_bkgnd*100:.4} %")
    
    tot_READOUT_PLANE = radiogenic_bkgnd.loc['READOUT_PLANE'].Total
    tot_CATHODE       = radiogenic_bkgnd.loc['CATHODE'].Total
    tot_FIELD_CAGE    = radiogenic_bkgnd.loc['FIELD_CAGE'].Total
    tot_ICS           = radiogenic_bkgnd.loc['INNER_SHIELDING', 'Total']
    print(f"\nTotal READOUT_PLANE background: {tot_READOUT_PLANE:8.4} Bq.  ->  {tot_READOUT_PLANE/tot_bkgnd*100:.4} %")
    print(f"Total CATHODE background:       {tot_CATHODE:8.4} Bq.  ->  {tot_CATHODE/tot_bkgnd*100:.4} %")
    print(f"Total FIELD_CAGE background:    {tot_FIELD_CAGE:8.4} Bq.  ->  {tot_FIELD_CAGE/tot_bkgnd*100:.4} %")
    print(f"Total ICS background:           {tot_ICS:8.4} Bq.  ->  {tot_ICS/tot_bkgnd*100:.4} %")


#### Radon contribution

Its final contribution comes from an extra Bi214 concentration in the central CATHODE

In [ ]:
# TO BE IMPLEMENTED

#### Muons contribution

Xe137 decay is the only muon contribution considered.

In [ ]:
# TO BE IMPLEMENTED

# LOADING & SELECTING REJECTION FACTORS

In [ ]:
rej_factors = get_rejection_factors(DETECTOR)
rej_factors

### OPTIONS TO STUDY

In [ ]:
energyRes  = 0.7
spatialDef = '10x10x10'

#### Signal efficiency

In [ ]:
sig_eff = rej_factors.loc[pd.IndexSlice['ACTIVE', energyRes, spatialDef], 'bb0nu']
sig_eff

#### Radiogenic rejection_factors

In [ ]:
radiogenic_rejections = rej_factors.loc[pd.IndexSlice[:, energyRes, spatialDef], ['Bi214', 'Tl208']]
radiogenic_rejections.drop('ACTIVE', level=0, axis=0, inplace=True)
radiogenic_rejections

#### Radon rejection_factor

As in previos NEXT detectors we assume that the whole amount of Radon contamination is translated to Bi214 contamination from the CATHODE.

In [ ]:
radon_rejection  = rej_factors.loc[pd.IndexSlice['CATHODE', energyRes, spatialDef], 'Bi214']
print(f"Radon rejection factor:       {radon_rejection:.4}")

#### Muons rejection_factors

The muons rejection factor = Xe137_activation * Xe137_rejection

In [ ]:
#Xe137_activation = XXXXX
Xe137_rejection = rej_factors.loc[pd.IndexSlice['ACTIVE', energyRes, spatialDef], 'Xe137']
#muon_rejection = Xe137_activation * Xe137_rejection

In [ ]:
if VERBOSE:
    print(f"Xe137 activation probability: XXXXX")
    print(f"Xe137 rejection factor:       {Xe137_rejection:.4}")
print(f"\nMuons rejection factor:       XXXXX")

# COMPUTING THE DETECTOR BACKGROUND INDEX

#### Estimating the Bq -> ckky conversion factor 

In [ ]:
secs_in_year    = 60 * 60 * 24 * 365
Xe136_abundance = 0.9026
Xe136_mass_kg   = det_dim['ACTIVE_mass'] * Xe136_abundance / units.kg
ROI_settings    = get_roi_settings(energyRes)

In [ ]:
toCKKY = secs_in_year / Xe136_mass_kg / ((ROI_settings['Emax'] - ROI_settings['Emin']) / units.keV)

In [ ]:
if VERBOSE:
    print(f"Xe136 mass: {Xe136_mass_kg:.4} kg.")
    print(f"ROI: [{ROI_settings['Emin']/units.keV:.4}, {ROI_settings['Emax']/units.keV:.4}] keV.")
    print(f"toCKKY factor: {toCKKY:.4}")

#### Radiogenic background index

In [ ]:
radiogenic_bkgnd_index_Bq = radiogenic_bkgnd[['Bi214', 'Tl208']] * radiogenic_rejections[['Bi214', 'Tl208']]
radiogenic_bkgnd_index_Bq['Total'] = radiogenic_bkgnd_index_Bq['Bi214'].add(radiogenic_bkgnd_index_Bq['Tl208'],
                                                                            fill_value = 0.)
radiogenic_bkgnd_index = radiogenic_bkgnd_index_Bq * toCKKY
radiogenic_bkgnd_index

In [ ]:
if VERBOSE:
    tot_bkgnd = radiogenic_bkgnd_index['Total'].sum()
    print(f"Radiogenic '{DETECTOR}' background index: {tot_bkgnd:8.3} ckky")
    
    tot_Bi = radiogenic_bkgnd_index['Bi214'].sum()
    tot_Tl = radiogenic_bkgnd_index['Tl208'].sum()
    print(f"\nTotal Bi214 background index: {tot_Bi:8.3} Bq.  ->  {tot_Bi/tot_bkgnd*100:.4} %")
    print(f"Total Tl208 background index: {tot_Tl:8.3} Bq.  ->  {tot_Tl/tot_bkgnd*100:.4} %")
    
    tot_READOUT_PLANE = radiogenic_bkgnd_index.loc[pd.IndexSlice['READOUT_PLANE', energyRes, spatialDef]].Total
    tot_CATHODE       = radiogenic_bkgnd_index.loc[pd.IndexSlice['CATHODE', energyRes, spatialDef]].Total
    tot_FIELD_CAGE    = radiogenic_bkgnd_index.loc[pd.IndexSlice['FIELD_CAGE', energyRes, spatialDef]].Total
    tot_ICS           = radiogenic_bkgnd_index.loc[pd.IndexSlice['INNER_SHIELDING', energyRes, spatialDef]].Total
    print(f"\nTotal READOUT_PLANE background index: {tot_READOUT_PLANE:8.3} ckky.  ->  {tot_READOUT_PLANE/tot_bkgnd*100:6.4} %")
    print(f"Total CATHODE       background index: {tot_CATHODE:8.3} ckky.  ->  {tot_CATHODE/tot_bkgnd*100:6.4} %")
    print(f"Total FIELD_CAGE    background index: {tot_FIELD_CAGE:8.3} ckky.  ->  {tot_FIELD_CAGE/tot_bkgnd*100:6.4} %")
    print(f"Total ICS           background index: {tot_ICS:8.3} ckky.  ->  {tot_ICS/tot_bkgnd*100:6.4} %")


#### Radon background index

In [ ]:
# TO BE IMPLEMENTED

#### Muons background index

In [ ]:
# TO BE IMPLEMENTED

# SENSITIVITY

In [ ]:
#from pybbsens import isotope